In [187]:
import pandas as pd
import  numpy as np
from sklearn import preprocessing 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score


In [188]:
dataset = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


In [189]:
dataset.head()

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,times_of_day,category,product,...,is_image,is_personalised,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience,click_rate
0,1,3,76,10439,39,5,1,Noon,6,26,...,0,0,0,0,0,0,0,0,14,0.103079
1,2,3,54,2570,256,5,1,Morning,2,11,...,0,0,0,0,0,0,0,0,10,0.700000
2,3,3,59,12801,16,5,1,Noon,2,11,...,1,0,1,0,0,0,0,0,16,0.002769
3,4,3,74,11037,30,4,0,Evening,15,9,...,0,0,0,0,0,0,0,0,10,0.010868
4,5,3,80,10011,27,5,1,Noon,6,26,...,0,0,1,0,0,0,0,0,14,0.142826


In [190]:
dataset.isna().sum()

campaign_id           0
sender                0
subject_len           0
body_len              0
mean_paragraph_len    0
day_of_week           0
is_weekend            0
times_of_day          0
category              0
product               0
no_of_CTA             0
mean_CTA_len          0
is_image              0
is_personalised       0
is_quote              0
is_timer              0
is_emoticons          0
is_discount           0
is_price              0
is_urgency            0
target_audience       0
click_rate            0
dtype: int64

In [191]:
dataset['times_of_day'].unique()

array(['Noon', 'Morning', 'Evening'], dtype=object)

In [192]:
label_encoder = preprocessing.LabelEncoder()
dataset['times_of_day']= label_encoder.fit_transform(dataset['times_of_day'])

In [193]:
test_df['times_of_day']= label_encoder.fit_transform(test_df['times_of_day'])

In [194]:
dataset.head()


,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,times_of_day,category,product,...,is_image,is_personalised,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience,click_rate
0,1,3,76,10439,39,5,1,2,6,26,...,0,0,0,0,0,0,0,0,14,0.103079
1,2,3,54,2570,256,5,1,1,2,11,...,0,0,0,0,0,0,0,0,10,0.700000
2,3,3,59,12801,16,5,1,2,2,11,...,1,0,1,0,0,0,0,0,16,0.002769
3,4,3,74,11037,30,4,0,0,15,9,...,0,0,0,0,0,0,0,0,10,0.010868
4,5,3,80,10011,27,5,1,2,6,26,...,0,0,1,0,0,0,0,0,14,0.142826


In [195]:
dataset.columns

Index(['campaign_id', 'sender', 'subject_len', 'body_len',
       'mean_paragraph_len', 'day_of_week', 'is_weekend', 'times_of_day',
       'category', 'product', 'no_of_CTA', 'mean_CTA_len', 'is_image',
       'is_personalised', 'is_quote', 'is_timer', 'is_emoticons',
       'is_discount', 'is_price', 'is_urgency', 'target_audience',
       'click_rate'],
      dtype='object')

In [196]:
X = dataset.drop(['campaign_id','is_timer','click_rate'] , axis = 1)
y = dataset.click_rate
Xt = test_df[X.columns]


In [197]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.033, random_state=0)
#X_train, X_test, y_train, y_test = train_test_split( train2[X.columns], train2.click_rate, test_size=0.033, random_state=0)
# feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
def select_features(X_train, y_train, X_test,k):
	# configure to select a subset of features
	fs = SelectKBest(score_func=f_regression,k=k)
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	# transform test input data
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs

X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test,5)


In [198]:
X_train_fs

array([[   55, 14959,    20,     9,     3],
       [   80, 10897,    21,    15,     1],
       [   97, 12876,    60,     2,     7],
       ...,
       [   67,  4827,    65,    15,     1],
       [  115, 19909,    22,    15,     9],
       [  129, 12895,    25,    10,     2]], dtype=int64)

In [199]:
regr = RandomForestRegressor( n_estimators= 100,  random_state=0)
#from sklearn.ensemble import ExtraTreesRegressor
#reg = ExtraTreesRegressor(n_estimators=100, random_state=0)
for i in range(1,19):
    X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test,i)
    regr.fit(X_train_fs,y_train)
    print(i,r2_score(regr.predict(X_test_fs),y_test))

1 0.5829709326214979
2 0.49854510352702297
3 0.6129087118815459
4 0.697920599769972
5 0.7522177471334682
6 0.6825485984779142
7 0.6284986901552085
8 0.4590517676944882
9 0.48375969785066963
10 0.5011866687769291
11 0.4544246325533078
12 0.3685689010778669
13 0.38805561168482783
14 0.4312351944334075
15 0.44489607365212314
16 0.4395195389090585
17 0.40972519665489826
18 0.4336203095142226


In [200]:
test_df_fs = fs.transform(test_df[X_train.columns])

In [201]:
# Import GridSearchCV
#from sklearn.model_selection import GridSearchCV

# Find the best parameters for the model
#parameters = {
#    'max_depth': [1,2,3,4,5,6,7,8,9,10],
#    'n_estimators': [100,200,300,400,500,600,700,800,900, 1000, 1100]
#}
#gridforest = GridSearchCV(regr, parameters, cv = 3, n_jobs = -1, verbose = 1)
#gridforest.fit(X_train_fs, y_train)
#gridforest.best_params_

In [202]:
X_train_fs

array([[    3,    55, 14959, ...,     0,     0,    12],
       [    3,    80, 10897, ...,     0,     0,    14],
       [    3,    97, 12876, ...,     0,     0,    16],
       ...,
       [    3,    67,  4827, ...,     0,     0,    12],
       [    3,   115, 19909, ...,     0,     1,    15],
       [    3,   129, 12895, ...,     0,     0,    12]], dtype=int64)

In [203]:
regr = RandomForestRegressor( n_estimators= 100, random_state=0)
regr.fit(X,y)
#reg = ExtraTreesRegressor(n_estimators=100, random_state=0).fit( X_train_fs, y_train)
predict = regr.predict(X)


In [204]:

#regr = svm.SVR()
#regr.fit(X_train_fs, y_train)
#predict = regr.predict(X_test_fs)

In [205]:


r2_score(y, predict)

0.9365180182279427

In [208]:
from sklearn.model_selection import cross_val_score

In [209]:
cross_val_score(reg,X,y,scoring='r2')

array([ 0.34017101,  0.39012745,  0.08601805,  0.19071526, -0.86357319])

In [210]:
test_pred = reg.predict(test_df_fs)

In [211]:
res = pd.DataFrame()
res['campaign_id'] = test_df.campaign_id
res['click_rate'] = test_pred

In [212]:
res.click_rate.describe()

count    762.000000
mean       0.043923
std        0.059831
min        0.000308
25%        0.011454
50%        0.021346
75%        0.051867
max        0.700000
Name: click_rate, dtype: float64

In [213]:
res.to_csv("raji_submission.csv",index=False)